# How likely is a Bones Day vs. a No Bones Day?

In [1]:
%load_ext lab_black

### Load Python tools

In [2]:
import pandas as pd
import altair as alt
import datetime as dt

In [3]:
today = dt.datetime.today()

### Read data collected from [TikTok](https://www.tiktok.com/@jongraz) about Noodle

In [4]:
src = pd.read_csv(
    "https://docs.google.com/spreadsheets/d/e/2PACX-1vQz8eEafXnIeKsNvGL8M56Ia6vu89JDCjfJ22ORl65So4kRYtmTbwmqLjE223fe4mjcZqkrs-KT6j8z/pub?\
gid=0&single=true&output=csv"
)

### Process dates

In [5]:
src["date"] = pd.to_datetime(src["date"])

In [6]:
df_date = pd.DataFrame()
index = pd.date_range(start="2021-09-01", end=today, freq="D")
df_date["date"] = index

### Merge the dates to backfill

In [7]:
df = pd.merge(src, df_date, on="date", how="outer")

In [8]:
df["weekday"] = df["date"].dt.day_name()
df["month"] = df["date"].dt.month_name()
df["weekend"] = df["date"].dt.day_name().isin(["Saturday", "Sunday"])

In [9]:
# df.drop(["url"], axis=1, inplace=True)

In [10]:
df.bones_nobones = df.bones_nobones.fillna("No video")

### What's our dataframe look like? 

In [11]:
df.head()

,date,bones_nobones,url,weekday,month,weekend
0,2021-11-19,No bones,https://www.tiktok.com/@jongraz/video/70323014...,Friday,November,False
1,2021-11-18,Bones!,https://www.tiktok.com/@jongraz/video/70319186...,Thursday,November,False
2,2021-11-16,Bones!,https://www.tiktok.com/@jongraz/video/70311878...,Tuesday,November,False
3,2021-11-15,Bones!,https://www.tiktok.com/@jongraz/video/70308085...,Monday,November,False
4,2021-11-12,No bones,https://www.tiktok.com/@jongraz/video/70297029...,Friday,November,False


### Percentage of days with Bones (only days with a video)

In [12]:
(
    df[df["bones_nobones"] != "No video"].value_counts("bones_nobones", normalize=True)
    * 100
).round(0)

bones_nobones
No bones    51.0
Bones!      49.0
dtype: float64

### Group by day type

In [13]:
days = (
    df.groupby(["bones_nobones"])
    .agg({"date": "count"})
    .reset_index()
    .rename(columns={"date": "count"})
)

### Simple bar chart explaining the count of day types

In [14]:
alt.Chart(days[days["bones_nobones"] != "No video"], title=" ").mark_bar().encode(
    y=alt.Y("bones_nobones", title="", axis=alt.Axis(tickCount=0, tickColor="#ffffff")),
    x=alt.X(
        "count",
        title="Days",
        axis=alt.Axis(grid=False, tickCount=6, tickColor="#ffffff"),
    ),
    color=alt.Color(
        "bones_nobones",
        title="",
        legend=None,
        scale=alt.Scale(
            domain=["Bones!", "No bones", "No video"],
            range=["#5ab4ac", "#d8b365", "#e6e6e6"],
        ),
    ),
).properties(width=500, height=70)

alt.Chart(...)

### Plot Noodle's days on a calendar

In [15]:
heatmap = (
    alt.Chart(df[df["date"] > "2021-08-31"], title=" ")
    .mark_rect()
    .encode(
        x=alt.X("date(date):O", title=" "),
        y=alt.Y("month(date):O", title=""),
        color=alt.Color(
            "bones_nobones",
            title="",
            scale=alt.Scale(
                domain=["Bones!", "No bones", "No video"],
                range=["#5ab4ac", "#d8b365", "#e6e6e6"],
            ),
        ),
        tooltip=[
            alt.Tooltip("monthdate(date):T", title="Date"),
            alt.Tooltip("bones_nobones", title="Tweets"),
        ],
    )
    .properties(width=500, height=70, title="Bones days since Sept. 1")
)

heatmap.configure_legend(orient="top", padding=10).configure_view(strokeOpacity=0)

alt.Chart(...)

### Prep for datawrapper 

In [16]:
df.head()

,date,bones_nobones,url,weekday,month,weekend
0,2021-11-19,No bones,https://www.tiktok.com/@jongraz/video/70323014...,Friday,November,False
1,2021-11-18,Bones!,https://www.tiktok.com/@jongraz/video/70319186...,Thursday,November,False
2,2021-11-16,Bones!,https://www.tiktok.com/@jongraz/video/70311878...,Tuesday,November,False
3,2021-11-15,Bones!,https://www.tiktok.com/@jongraz/video/70308085...,Monday,November,False
4,2021-11-12,No bones,https://www.tiktok.com/@jongraz/video/70297029...,Friday,November,False


In [17]:
df["bones_class"] = (
    df["bones_nobones"]
    .str.lower()
    .str.replace(" ", "-", regex=False)
    .str.replace("!", "", regex=False)
)

In [18]:
df["weekend_class"] = (
    df["weekend"]
    .astype(str)
    .str.lower()
    .str.replace(" ", "-", regex=False)
    .str.replace("!", "", regex=False)
)

In [19]:
grouped_all = (
    df.groupby("bones_nobones")
    .agg({"date": "size"})
    .reset_index()
    .rename(columns={"date": "Days"})
)

In [20]:
grouped_all

,bones_nobones,Days
0,Bones!,20
1,No bones,21
2,No video,43


In [21]:
grouped_all.iloc[1, 1]

21

In [22]:
grouped_weekends = (
    df.groupby(["bones_nobones", "weekend"])
    .agg({"date": "size"})
    .reset_index()
    .rename(columns={"date": "Days"})
)

In [23]:
grouped_weekends

,bones_nobones,weekend,Days
0,Bones!,False,17
1,Bones!,True,3
2,No bones,False,16
3,No bones,True,5
4,No video,False,27
5,No video,True,16


In [24]:
grouped_weekends.iloc[3, 2]

5

In [25]:
stacked_bar = pd.DataFrame(
    {
        "Category": ["All days", "Weekends", "Weekdays"],
        "Bones": [
            grouped_all.iloc[0, 1],
            grouped_weekends.iloc[1, 2],
            grouped_weekends.iloc[0, 2],
        ],
        "No Bones": [
            grouped_all.iloc[1, 1],
            grouped_weekends.iloc[3, 2],
            grouped_weekends.iloc[2, 2],
        ],
    }
)

In [26]:
stacked_bar

,Category,Bones,No Bones
0,All days,20,21
1,Weekends,3,5
2,Weekdays,17,16


### Export

In [27]:
df["display_date"] = df["date"].dt.strftime("%b. %-d ")
df["date"] = df["date"].astype(str)

In [28]:
stacked_bar.to_csv("../_data/bonesdays_summary_csv.csv", index=False)
df[df["bones_nobones"] != "No video"].to_csv("../_data/bonesdays_csv.csv", index=False)

In [29]:
stacked_bar.to_json("../_data/bonesdays_summary.json", indent=4, orient="records")
df[df["bones_nobones"] != "No video"].to_json(
    "../_data/bonesdays.json", indent=4, orient="records"
)